In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.optimize import minimize
from IPython.display import display, Latex

plt.style.use(
    "https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle"
)

In [2]:
def log_likelihood_computation(returns, params):
    theta, kappa, sigma,rho, mu = [param for param in params.values()]
    residuals = returns - mu
    expected_variance = ((returns - mu) ** 2) + kappa * (theta - (returns - mu)**2)
    log_likelihood_part1 =  1 / (2 * np.pi * np.sqrt(expected_variance) * sigma * np.sqrt(1 - rho**2))
    log_likelihood_part2 = np.exp(-((((residuals / np.sqrt(expected_variance))**2) 
                                    - 2 * rho * (residuals / np.sqrt(expected_variance)) * 
                                    (((residuals**2 - expected_variance) / sigma)) + 
                                    (((residuals**2 - expected_variance) / sigma)**2)) / (2 * (1 - rho**2))))
    log_likelihood = np.log(log_likelihood_part1 * log_likelihood_part2)
    return log_likelihood

def log_likelihood(x, returns):
    theta, kappa, sigma,rho, mu = [param for param in x]
    
    lg = np.sum(log_likelihood_computation(returns, {"theta": theta, "kappa": kappa, "sigma": sigma, "rho": rho, "mu": mu}))
    
    return -lg

In [3]:
returns_raw = pd.read_excel(r'C:\Users\rrenard\OneDrive - Arkus\Desktop\spx.xlsx')

In [4]:
returns = np.log(returns_raw['SPX'] / returns_raw['SPX'].shift(1)) *100

In [5]:
returns_array = np.array(returns)
# drop nan values
returns_array = returns_array[1:]

In [79]:
params = {"theta": {"x0": 0.682, "lbud": [1e-8,None]},
          "kappa": {"x0": 1, "lbud": [1e-8,None]},
          "sigma": {"x0": 1.130, "lbud": [1e-8,None]},
          "rho": {"x0": 0, "lbud": [-1,1]},
          "mu": {"x0": 0.095, "lbud": [1e-8,None]}
          }

x0 = [param["x0"] for key, param in params.items()]
bnds = [params['lbud'] for key, params in params.items()]

In [80]:
result = minimize(log_likelihood, x0, args=(returns_array), bounds=bnds, method='SLSQP')

C:\Users\rrenard\AppData\Local\Temp\ipykernel_11052\3109690707.py:5: RuntimeWarning: divide by zero encountered in divide
  log_likelihood_part1 =  1 / (2 * np.pi * np.sqrt(expected_variance) * sigma * np.sqrt(1 - rho**2))
C:\Users\rrenard\AppData\Local\Temp\ipykernel_11052\3109690707.py:6: RuntimeWarning: divide by zero encountered in divide
  log_likelihood_part2 = np.exp(-((((residuals / np.sqrt(expected_variance))**2)
C:\Users\rrenard\AppData\Local\Temp\ipykernel_11052\3109690707.py:10: RuntimeWarning: invalid value encountered in multiply
  log_likelihood = np.log(log_likelihood_part1 * log_likelihood_part2)


In [81]:
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1673.8123376556182
       x: [ 1.322e-05  1.518e-05  2.635e-04 -7.272e-01  1.025e-08]
     nit: 11
     jac: [-1.182e+01  2.833e+05  9.400e+05 -4.470e+02  9.316e+02]
    nfev: 56
    njev: 7

In [82]:
result.x

array([ 1.32178722e-05,  1.51780572e-05,  2.63519816e-04, -7.27186658e-01,
        1.02464788e-08])

In [83]:
theta_hat = r'$\hat{\theta} = '+str(result.x[0])+'$'
kappa_hat = r'$\hat{\kappa} = '+str(result.x[1])+'$'
sigma_hat = r'$\hat{\sigma} = '+str(result.x[2])+'$'
rho_hat = r'$\hat{\rho} = '+str(result.x[3])+'$'
mu_hat = r'$\hat{\mu} = '+str(result.x[4])+'$'

In [84]:
display(Latex(theta_hat))
display(Latex(kappa_hat))
display(Latex(sigma_hat))
display(Latex(rho_hat))
display(Latex(mu_hat))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>